In [ ]:
!git clone https://github.com/chakki-works/chABSA-dataset.git

Cloning into 'chABSA-dataset'...
remote: Enumerating objects: 187, done.
remote: Total 187 (delta 0), reused 0 (delta 0), pack-reused 187
Receiving objects: 100% (187/187), 1.89 MiB | 5.68 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
pip install pandas scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip chABSA-dataset.zip

--2023-04-17 16:18:37--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.9.20, 52.219.200.12, 52.219.152.160, ...
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.9.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K  3.70MB/s    in 0.2s    

2023-04-17 16:18:38 (3.70 MB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflat

In [ ]:
data_path = "chABSA-dataset"

In [ ]:
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

def load_data(path):
    sentences = []
    labels = []

    for file in os.listdir(path):
        if file.endswith(".json"):
            file_path = os.path.join(path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError:
                    print(f"Error decoding file: {file}")
                    continue

                for sent in data["sentences"]:
                    if len(sent["opinions"]) > 0:
                        sentences.append(sent["sentence"])
                        label = sent["opinions"][0]["polarity"]
                        labels.append(label)

    return sentences, labels

data_path = "chABSA-dataset"
X, y = load_data(data_path)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LinearSVC()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negative       0.64      0.49      0.56       269
     neutral       0.60      0.11      0.19        27
    positive       0.65      0.80      0.72       347

    accuracy                           0.64       643
   macro avg       0.63      0.47      0.49       643
weighted avg       0.64      0.64      0.63       643



In [ ]:
# Google Colabでモデルとベクトル化器を保存する
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# ローカル環境にダウンロードする
from google.colab import files

files.download('model.pkl')
files.download('vectorizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>